In [1]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import gym

In [ ]:
env = gym.make('CartPole-v0')

In [ ]:
state_shape = env.observation_space.shape[0]

In [ ]:
gamma = 0.95

In [ ]:
def discount_and_normalize_rewards(episode_rewards):
    discounted_rewards = np.zeros_like(episode_rewards)
    
    reward_to_go = 0.0
    for i in reversed(len(episode_rewards)):
        reward_to_go = reward_to_go * gamma * episode_rewards[i]
        discounted_rewards[i] = reward_to_go
    
    # normalize and return the reward
    discounted_rewards -= np.mean(discounted_rewards)
    discounted_rewards /= np.std(discounted_rewards)
    
    return discounted_rewards